In [7]:
from qiskit import *
from qiskit_ibm_provider import IBMProvider
# import qiskit_ibm_provider.jupyter

In [8]:
%ibm_quantum_dashboard

UsageError: Line magic function `%ibm_quantum_dashboard` not found.


In [10]:
provider = IBMProvider()

AccountNotFoundError: 'No default ibm_quantum account saved.'

In [11]:
from qiskit_ibm_provider import least_busy

backend = least_busy(
    provider.backends(
        simulator=False, filters=lambda b: b.configuration().n_qubits >= 5
    )
)

qc.QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

mapped_circuit = transpile(qc, backend=backend)
job = backend.run(mapped_circuit, shots=1024)


NameError: name 'provider' is not defined